In [1]:
import Ipynb_importer
from a_basic_quant import *
from b_model import *
from c_train_and_test import *
from d_post_training_quantize import *

importing Jupyter notebook from a_basic_quant.ipynb
importing Jupyter notebook from b_model.ipynb
importing Jupyter notebook from c_train_and_test.ipynb
importing Jupyter notebook from d_post_training_quantize.ipynb


In [2]:
import torch

In [3]:
a = torch.randn((64,1,28,28))
a.shape

torch.Size([64, 1, 28, 28])

In [4]:
conv = torch.nn.Conv2d(1, 40, 3, 1)
res = conv(a)

In [5]:
res.shape

torch.Size([64, 40, 26, 26])

In [6]:
max_val =[float(res[:,i,:,:].max() )for i in range(len(res[0]))] 
max_val[:5]


[2.372403621673584,
 2.511977434158325,
 1.831723928451538,
 2.1057021617889404,
 2.2107653617858887]

In [7]:
min_val = [float(res[:,i,:,:].min() )for i in range(len(res[0]))] 
min_val[:5]

[-2.066378355026245,
 -2.436711311340332,
 -2.116239547729492,
 -1.4519824981689453,
 -2.306870698928833]

In [8]:
scale = []
zero_point = []
for i in range(len(max_val)):
    s, z = calcu_scale_and_zeropoint(min_val[i], max_val[i], 8)
    scale.append(s)
    zero_point.append(z)
    
scale[:5]
zero_point[:5]

[118.0, 125.0, 136.0, 104.0, 130.0]

In [9]:
def quantize_tensor(x:list, scale:list, zero_point:list, num_bits=8):
    q_min = 0.
    q_max = 2. ** num_bits -1.
    
    q_x = torch.zeros(x.shape)
    for i in range(len(scale)):
        q_x[:,i,:,:] = x[:,i,:,:] /scale[i] + zero_point[i]
        q_x[:,i,:,:].clamp_(0, 255).round()
        
    return q_x.int()

def dequantize_tensor(q_x, scale, zero_point):
    x = torch.zeros(q_x.shape)
    for i in range(len(scale)):
        x[:,i,:,:] = scale[i] * (q_x[:,i,:,:] - zero_point[i])
    
    return x

In [10]:
q_res = quantize_tensor(res, scale, zero_point)
q_res[0][0][0][0:5]

tensor([122, 135, 110, 122, 112], dtype=torch.int32)

In [11]:
dq_res = dequantize_tensor(q_res, scale, zero_point)
dq_res[0][0:5][0][0]

tensor([ 0.0696,  0.2959, -0.1393,  0.0696, -0.1044, -0.9574, -0.1915,  0.1741,
         0.7311,  0.6789,  0.7659,  0.4004,  0.6789, -0.9052, -0.8355, -0.2089,
         0.2785,  0.4700, -0.5918,  0.0174,  0.1567,  0.2611, -0.1044,  0.2611,
         0.0522, -0.0696])

In [12]:
res[0][0:5][0][0]

tensor([ 0.0818,  0.3110, -0.1340,  0.0779, -0.0960, -0.9421, -0.1799,  0.1898,
         0.7397,  0.6930,  0.7696,  0.4015,  0.6838, -0.9022, -0.8329, -0.1941,
         0.2952,  0.4715, -0.5874,  0.0283,  0.1636,  0.2781, -0.0994,  0.2707,
         0.0542, -0.0564], grad_fn=<SelectBackward>)

In [13]:
scale = lambda max_val, min_val :  float((max_val - min_val) / (255))
s = scale(255,2)
s

0.9921568627450981

In [14]:
calcu_zero_point = lambda max_val, min_val :  np.clip(int(255 - max_val / s), 0,255)
z = calcu_zero_point(16,2)
z

238

In [15]:
calcu_scale =  lambda max_val, min_val : float((max_val - min_val) / (q_max - q_min))
calcu_zero_point = lambda max_val, scale :  np.clip(int(q_max - max_val / scale), q_min, q_max)

q_max = 255
q_min = 0
scale = calcu_scale(64,-23)
scale
zero_point = calcu_zero_point(64, scale)
zero_point

67

In [16]:
def calcu_scale_and_zeropoint(min_val, max_val, num_bits=8, is_per_channel_quantize=False):
    # 计算 scale 和 zero_point 的基本公式
    # is_per_channel_quantize 决定是按照逐通道量化还是按照逐层量化来计算 scale 和 zero_point 
    q_min = 0.
    q_max = 2. ** num_bits - 1
    
    # 匿名函数，计算 scale 和 zero_point
    calcu_scale =  lambda max_val, min_val : float((max_val - min_val) / (q_max - q_min))
    calcu_zero_point = lambda max_val, scale :  np.clip(int(q_max - max_val / scale), q_min, q_max)
    
    # 是否是逐通道量化
    if  is_per_channel_quantize:
        scale = []
        zero_point = []

        for i in range(len(min_val)):
            scale.append(calcu_scale(max_val[i], min_val[i]))
            zero_point.append( calcu_zero_point(max_val[i], scale[i]))  
            
    else:
        scale = calcu_scale(max_val, min_val)
        zero_point = calcu_zero_point(max_val, scale)
        
    return scale, zero_point


In [17]:
scale, zero_point = calcu_scale_and_zeropoint(-23, 64)
print(scale, zero_point)

0.3411764705882353 67.0


In [18]:
scale, zero_point = calcu_scale_and_zeropoint(min_val, max_val, 8, True)
print(scale, zero_point)

[0.017406988143920898, 0.019406622531367282, 0.015482209710513844, 0.013951704548854454, 0.017716219845940084, 0.025020348791982613, 0.018590784072875975, 0.021531027438593846, 0.015164483762254903, 0.018993748870550418, 0.01880497651941636, 0.021872305402568744, 0.016954759522980336, 0.016447824590346393, 0.016811740164663278, 0.022289041444367053, 0.012003157185573203, 0.019158273584702436, 0.01622291873483097, 0.0213474245632396, 0.01177502192702948, 0.015453707470613368, 0.013611221780963972, 0.011446661107680377, 0.020656879275452858, 0.01838410321403952, 0.019261037602144128, 0.01818844570833094, 0.02404925028483073, 0.015078667098400639, 0.022204766553990982, 0.018716723311181162, 0.02474205540675743, 0.01698904504962996, 0.015915822982788087, 0.018676993426154642, 0.022611205718096564, 0.01764062245686849, 0.016676984113805433, 0.023288680057899625] [118.0, 125.0, 136.0, 104.0, 130.0, 120.0, 143.0, 115.0, 147.0, 134.0, 144.0, 156.0, 116.0, 124.0, 109.0, 126.0, 153.0, 140.0, 122

In [19]:
def quantize_tensor(x, scale, zero_point, num_bits=8, is_per_channel_quantize=False ):
    # 计算 scale 和 zero_point 的基本公式
    # is_per_channel_quantize 决定是按照逐通道量化还是按照逐层量化
    q_min = 0.
    q_max = 2. ** num_bits - 1
    
    if is_per_channel_quantize:
        q_x = torch.zeros(x.shape)
        for i in range(len(scale)):
            q_x[:,i,:,:] = x[:,i,:,:] /scale[i] + zero_point[i]
            q_x[:,i,:,:].clamp_(0, 255).round()
            
    else:
        q_x = x / scale + zero_point
        q_x.clamp_(q_min, q_max).round()  # q=round(r/S+Z)
        
    return q_x.int()

def dequantize_tensor(q_x, scale, zero_point, is_per_channel_quantize=False ):
    
    if is_per_channel_quantize:
        
        x = torch.zeros(q_x.shape)
        for i in range(len(scale)):
            x[:,i,:,:] = scale[i] * (q_x[:,i,:,:] - zero_point[i])
            
    else:
        x = scale * (q_x -zero_point)
        
    return x

In [20]:
q_res = quantize_tensor(res, scale, zero_point, 8, True)
q_res[0][0][0][0:5]

tensor([122, 135, 110, 122, 112], dtype=torch.int32)

In [21]:
dq_res = dequantize_tensor(q_res, scale, zero_point, True)
dq_res[0][0:5][0][0]

tensor([ 0.0696,  0.2959, -0.1393,  0.0696, -0.1044, -0.9574, -0.1915,  0.1741,
         0.7311,  0.6789,  0.7659,  0.4004,  0.6789, -0.9052, -0.8355, -0.2089,
         0.2785,  0.4700, -0.5918,  0.0174,  0.1567,  0.2611, -0.1044,  0.2611,
         0.0522, -0.0696])

In [22]:
res[0][0:5][0][0]

tensor([ 0.0818,  0.3110, -0.1340,  0.0779, -0.0960, -0.9421, -0.1799,  0.1898,
         0.7397,  0.6930,  0.7696,  0.4015,  0.6838, -0.9022, -0.8329, -0.1941,
         0.2952,  0.4715, -0.5874,  0.0283,  0.1636,  0.2781, -0.0994,  0.2707,
         0.0542, -0.0564], grad_fn=<SelectBackward>)

In [23]:
max_layer = res.max()
min_layer = res.min()

In [24]:
scale_layer, zero_point_layer = calcu_scale_and_zeropoint(min_layer, max_layer, 8, False)
print(scale_layer, zero_point_layer)

0.026581289246678352 128.0


In [25]:
q_res_layer = quantize_tensor(res, scale_layer, zero_point_layer, 8, False)
q_res_layer[0][0][0][0:5]

tensor([131, 139, 122, 130, 124], dtype=torch.int32)

In [26]:
dq_res_layer = dequantize_tensor(q_res_layer, scale_layer, zero_point_layer, False)
dq_res_layer[0][0:5][0][0]

tensor([ 0.0797,  0.2924, -0.1595,  0.0532, -0.1063, -0.9569, -0.1861,  0.1861,
         0.7177,  0.6911,  0.7443,  0.3987,  0.6645, -0.9038, -0.8506, -0.2127,
         0.2924,  0.4519, -0.6114,  0.0266,  0.1595,  0.2658, -0.1063,  0.2658,
         0.0532, -0.0797])

In [40]:
def calcu_max_and_min(x, max_val, min_val, is_per_channel_quantize=False ):
    calcu_max = lambda x, max_val: max(0, x.max()) if max_val is None \
                                                                else max(0, x.max(), max_val)
    calcu_min = lambda x, min_val: min(0, x.min()) if min_val is None \
                                                                else min(0, x.min(), min_val)
    if is_per_channel_quantize:
#         max_val =[calcu_max(x[:,i,:,:], max_val[i]) for i in range(len(x[0]))]    
#         min_val =[calcu_min(x[:,i,:,:], min_val[i]) for i in range(len(x[0]))]    这样写也行，不过读起来差一点
        new_max_val = []
        new_min_val = []
        for i in range(len(x[0])):
            if max_val == None or  max_val[i] == None:
                new_max_val.append(calcu_max(x[:,i,:,:], None))
            else:
                new_max_val.append(calcu_max(x[:,i,:,:], max_val[i]))
            if min_val == None or  min_val[i] == None:
                new_min_val.append(calcu_min(x[:,i,:,:], None))
            else:
                new_min_val.append(calcu_min(x[:,i,:,:], min_val[i]))
            
    else:
        new_max_val = calcu_max(x, max_val)
        new_min_val = calcu_min(x, min_val)
        
    return new_min_val, new_max_val

In [41]:
min_layer, max_layer = calcu_max_and_min(res, None, None, False)
print(min_layer, max_layer)

tensor(-3.4151, grad_fn=<MinBackward1>) tensor(3.3631, grad_fn=<MaxBackward1>)


In [42]:
min_channel, max_channel = calcu_max_and_min(res, None, None, True)
print(min_channel, max_channel)

[tensor(-2.0664, grad_fn=<MinBackward1>), tensor(-2.4367, grad_fn=<MinBackward1>), tensor(-2.1162, grad_fn=<MinBackward1>), tensor(-1.4520, grad_fn=<MinBackward1>), tensor(-2.3069, grad_fn=<MinBackward1>), tensor(-3.0171, grad_fn=<MinBackward1>), tensor(-2.6649, grad_fn=<MinBackward1>), tensor(-2.4907, grad_fn=<MinBackward1>), tensor(-2.2375, grad_fn=<MinBackward1>), tensor(-2.5613, grad_fn=<MinBackward1>), tensor(-2.7123, grad_fn=<MinBackward1>), tensor(-3.4151, grad_fn=<MinBackward1>), tensor(-1.9707, grad_fn=<MinBackward1>), tensor(-2.0458, grad_fn=<MinBackward1>), tensor(-1.8461, grad_fn=<MinBackward1>), tensor(-2.8166, grad_fn=<MinBackward1>), tensor(-1.8463, grad_fn=<MinBackward1>), tensor(-2.6948, grad_fn=<MinBackward1>), tensor(-1.9904, grad_fn=<MinBackward1>), tensor(-2.7237, grad_fn=<MinBackward1>), tensor(-1.2434, grad_fn=<MinBackward1>), tensor(-1.9492, grad_fn=<MinBackward1>), tensor(-2.0022, grad_fn=<MinBackward1>), tensor(-1.4950, grad_fn=<MinBackward1>), tensor(-2.4877,

In [43]:
scale_channel, zero_point_channel = calcu_scale_and_zeropoint(min_channel, max_channel, 8, True)
print(scale_channel, zero_point_channel)

[0.01740698702633381, 0.019406622275710106, 0.015482209622859955, 0.01395170483738184, 0.017716221511363983, 0.02502034790813923, 0.018590785562992096, 0.02153102681040764, 0.015164483338594437, 0.018993748351931572, 0.01880497671663761, 0.021872306242585182, 0.016954759135842323, 0.016447825357317924, 0.016811741515994072, 0.022289041429758072, 0.012003157287836075, 0.019158273935317993, 0.016222918406128883, 0.021347424015402794, 0.011775022372603416, 0.015453707426786423, 0.013611221686005592, 0.011446661315858364, 0.020656879991292953, 0.01838410273194313, 0.019261037930846214, 0.018188446760177612, 0.024049250409007072, 0.015078667551279068, 0.022204766049981117, 0.0187167227268219, 0.02474205568432808, 0.016989046707749367, 0.015915822237730026, 0.018676992505788803, 0.022611206397414207, 0.017640622332692146, 0.01667698472738266, 0.023288680240511894] [118.0, 125.0, 136.0, 104.0, 130.0, 120.0, 143.0, 115.0, 147.0, 134.0, 144.0, 156.0, 116.0, 124.0, 109.0, 126.0, 153.0, 140.0, 12

In [44]:
q_res_channel = quantize_tensor(res, scale_channel, zero_point_channel, 8, True)
q_res_channel[0][0][0][0:5]

tensor([122, 135, 110, 122, 112], dtype=torch.int32)

In [46]:
dq_res_channel = dequantize_tensor(q_res_channel, scale_channel, zero_point_channel,True)
dq_res_channel[0][0:5][0][0]

tensor([ 0.0696,  0.2959, -0.1393,  0.0696, -0.1044, -0.9574, -0.1915,  0.1741,
         0.7311,  0.6789,  0.7659,  0.4004,  0.6789, -0.9052, -0.8355, -0.2089,
         0.2785,  0.4700, -0.5918,  0.0174,  0.1567,  0.2611, -0.1044,  0.2611,
         0.0522, -0.0696])

In [47]:
res[0][0:5][0][0]

tensor([ 0.0818,  0.3110, -0.1340,  0.0779, -0.0960, -0.9421, -0.1799,  0.1898,
         0.7397,  0.6930,  0.7696,  0.4015,  0.6838, -0.9022, -0.8329, -0.1941,
         0.2952,  0.4715, -0.5874,  0.0283,  0.1636,  0.2781, -0.0994,  0.2707,
         0.0542, -0.0564], grad_fn=<SelectBackward>)